In [1]:
from shapely.geometry import Point
from pymongo import MongoClient
from shapely.geometry import Point, Polygon


def connect_to_mongodb(database_name, collection_name):
    """
    Connects to MongoDB and returns the MongoDB client and collection.
    """
    client = MongoClient('localhost', 27017)
    db = client[database_name]
    collection = db[collection_name]
    return client, collection

def retrieve_data_from_mongodb(collection):
    """
    Retrieves GeoJSON data from MongoDB collection.
    """
    cursor = collection.find()
    geojson_data = {"type": "FeatureCollection", "features": []}

    for document in cursor:
        geojson_data["features"].append(document)

    return geojson_data

def coordinates_to_polygon(coordinates):
    """
    Converts coordinates to a Shapely Polygon.
    """
    return Polygon(coordinates)

def get_postal_code_for_point(latitude, longitude, postal_code_polygons):
    """
    Returns the postal code for a point specified by latitude and longitude.
    """
    # Create a Point object for the specified latitude and longitude
    point = Point(longitude, latitude)
    
    # Check each postal code polygon to see if it contains the point
    for postal_code, polygon in postal_code_polygons.items():
        if polygon.contains(point):
            # If the polygon contains the point, return its postal code
            return postal_code
    
    # If no polygon contains the point, return None
    return None

# Example usage
if __name__ == "__main__":
    # MongoDB configuration
    database_name = "ZIP_Poly_Ger_Test"
    collection_name = "ZIP_Poly_Ger_Collection_Test"

    # Connect to MongoDB
    mongo_client, mongo_collection = connect_to_mongodb(database_name, collection_name)

    # Retrieve GeoJSON data from MongoDB
    geojson_data_from_mongo = retrieve_data_from_mongodb(mongo_collection)

    # Load GeoJSON data from MongoDB for comparison
    postal_code_polygons_mongo = {document.get('postal_code'): coordinates_to_polygon(document.get('geometry')['coordinates'][0]) for document in geojson_data_from_mongo['features']}

    # Test the get_postal_code_for_point function
    latitude = 51.316669
    longitude = 9.500000
    postal_code = get_postal_code_for_point(latitude, longitude, postal_code_polygons_mongo)
    print(f"The postal code for the point ({latitude}, {longitude}) is: {postal_code}")

    # Close MongoDB connection when done
    mongo_client.close()


The postal code for the point (51.316669, 9.5) is: 34117
